# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

boston = datasets.load_boston()

def populate_table(tablename, feature_names):
    X = boston.data  
    y = boston.target
    N = X.shape[0]
    y = y.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : list(boston.feature_names),
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/boston.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/boston.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TGT
230,230,230,0.53700,0.0,6.20,0,0.504,5.981,68.1,3.6715,8,307,17.4,378.35,11.65,24
112,112,112,0.12329,0.0,10.01,0,0.547,5.913,92.9,2.3534,6,432,17.8,394.95,16.21,18
125,125,125,0.16902,0.0,25.65,0,0.581,5.986,88.4,1.9929,2,188,19.1,385.02,14.81,21
9,9,9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18
213,213,213,0.14052,0.0,10.59,0,0.489,6.375,32.3,3.9454,4,277,18.6,385.81,9.38,28
109,109,109,0.26363,0.0,8.56,0,0.520,6.229,91.2,2.5451,5,384,20.9,391.23,15.55,19
127,127,127,0.25915,0.0,21.89,0,0.624,5.693,96.0,1.7883,4,437,21.2,392.11,17.19,16
244,244,244,0.20608,22.0,5.86,0,0.431,5.593,76.5,7.9549,7,330,19.1,372.49,12.50,17
406,406,406,20.71620,0.0,18.10,0,0.659,4.138,100.0,1.1781,24,666,20.2,370.22,23.34,11
490,490,490,0.20746,0.0,27.74,0,0.609,5.093,98.0,1.8226,4,711,20.1,318.43,29.68,8


## Training a Model

In [5]:
import torch
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetRegressor

torch.manual_seed(1234)

torch.set_default_tensor_type('torch.DoubleTensor')

num_units = 15
# Example of using Sequential
model = nn.Sequential(
    nn.Linear(13, num_units),
    nn.Dropout(),
    nn.Linear(num_units , num_units * 3),
    nn.Dropout(),
    nn.Linear(num_units * 3, 1))

clf = NeuralNetRegressor(
    model,
    max_epochs=10,
)

import numpy as np
X = df[metadata['features']].astype(np.double).values
X = (X - X.mean(axis = 0)) / X.std(axis = 0)
y = df[metadata['targets']].astype(np.double).values
y = (y - y.mean(axis = 0)) / y.std(axis = 0)

clf.fit(X, y)


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.4840        0.3676  0.0383
      2        1.0398        0.3277  0.0209
      3        0.9010        0.3200  0.0198
      4        0.8858        0.3147  0.0198
      5        0.7486        0.3112  0.0198
      6        0.7510        0.3118  0.0198
      7        0.7595        0.3120  0.0213


/home/antoine/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


      8        0.7289        0.3043  0.0206
      9        0.7112        0.2972  0.0363
     10        0.7292        0.2939  0.0253


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Sequential(
    (0): Linear(in_features=13, out_features=15, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=15, out_features=45, bias=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=45, out_features=1, bias=True)
  ),
)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	namespace layer_0 {
	
		std::vector<std::string> get_input_names(){
			std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3", "Feature_4", "Feature_5", "Feature_6", "Feature_7", "Feature_8", "Feature_9", "Feature_10", "Feature_11", "Feature_12" };
	
			return lFeatures;
		}
	
		std::vector<std::string> get_output_names(){
			std::vector<std::string> lOutputs = { "output_0", "output_1", "output_2", "output_3", "output_4", "output_5", "output_6", "output_7", "output_8", "output_9", "output_10", "output_11", "output_12", "output_13", "output_14" };
	
			return lOutputs;
		}
	
		tTable compute_features(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3, std::any Feature_4, std::any Feature_5, std::any Feature_6, std::any Feature_7, std::any Feature_8, std::any Feature_9, std::any Feature_10, std::any Feature_11, std::any Feature_12) {
	
			tTable lTable;
	
			lTable["output_0"] = { -0.2504802628970162 + -0.14609

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:100])
        print(result2[-100:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/boston2.csv" , ["Feature_" + str(i) for i,x in enumerate(metadata["features"])])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/boston2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_139678602050672.exe /tmp/sklearn2sql_cpp_139678602050672.cpp'
idx,Estimator
0,-28.65289016575531
1,-20.98570689417278
2,-21.13010742393421
3,-17.77532006997907
4,
9114010
502,-26.96366667936048
503,-25.94445027410062
504,-26.28048146819766
505,-26.66797847150204



In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Estimator
230,230,-33.356720
112,112,-53.543854
125,125,-14.483438
9,9,-32.945750
213,213,-29.425179
109,109,-45.730414
127,127,-55.703568
244,244,-36.153190
406,406,-95.661881
490,490,-106.885336


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(clf.predict(X), columns=['Estimator']);
skl_output = pd.concat([skl_output_key, skl_output_score] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Estimator
230,230,-33.356720
112,112,-53.543854
125,125,-14.483438
9,9,-32.945750
213,213,-29.425179
109,109,-45.730414
127,127,-55.703568
244,244,-36.153190
406,406,-95.661881
490,490,-106.885336


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Estimator_skl,idx_cpp,Estimator_cpp
230,230,-33.356720,230,-33.356720
112,112,-53.543854,112,-53.543854
125,125,-14.483438,125,-14.483438
9,9,-32.945750,9,-32.945750
213,213,-29.425179,213,-29.425179
109,109,-45.730414,109,-45.730414
127,127,-55.703568,127,-55.703568
244,244,-36.153190,244,-36.153190
406,406,-95.661881,406,-95.661881
490,490,-106.885336,490,-106.885336


In [14]:
lDiff = cpp_skl_join['Estimator_skl'] - cpp_skl_join['Estimator_cpp']


In [15]:
lDiff.describe()

count    5.060000e+02
mean     5.616939e-17
std      1.474928e-14
min     -5.684342e-14
25%     -7.105427e-15
50%      0.000000e+00
75%      7.105427e-15
max      5.684342e-14
dtype: float64